<a href="https://colab.research.google.com/github/Ricardo-HJ/Datathon-2024/blob/main/vuelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from scipy import stats
import joblib

Cargar datos

In [2]:
# Cargar el DataFrame
df_vuelos = pd.read_csv('Filghts TEC_Valid.csv')

Eliminar registros nan

In [3]:
# Filtrar datos eliminando registros con valores nulos en las columnas especificadas
df_vuelos = df_vuelos.dropna(subset=['Aeronave', 'DepartureStation', 'ArrivalStation', 'Destination_Type', 'Origin_Type'])

# Eliminar columna 'Flight_ID'
df_vuelos.drop(columns=['Flight_ID'], inplace=True)

Ceación de duración vuelos y fecha/hora vuelos

In [4]:
# Convertir la columna 'STD' a formato datetime
df_vuelos['STD'] = pd.to_datetime(df_vuelos['STD'])
df_vuelos['STA'] = pd.to_datetime(df_vuelos['STA'])
df_vuelos['Departure_WeekDay'] = df_vuelos['STD'].dt.day_name()

# Extraer la hora y el mes de la columna 'STD'
df_vuelos['Departure_Hour'] = df_vuelos['STD'].dt.hour
df_vuelos['Arrival_Hour'] = df_vuelos['STA'].dt.hour
df_vuelos['Departure_Month'] = df_vuelos['STD'].dt.month
# Calcular la duración del vuelo restando STA de STD
df_vuelos['Flight_Time'] = (pd.to_datetime(df_vuelos['STA']) - pd.to_datetime(df_vuelos['STD'])).dt.total_seconds() / 60

# Convertir el tiempo de vuelo a entero
df_vuelos['Flight_Time'] = df_vuelos['Flight_Time'].astype(int)
# Convertir la hora de salida y el mes a entero
df_vuelos['Departure_Hour'] = df_vuelos['Departure_Hour'].astype(int)
df_vuelos['Departure_Month'] = df_vuelos['Departure_Month'].astype(int)

Eliminar columnas innecesarias

In [5]:
# Eliminar columnas 'STD', 'STA' y 'Bookings'
df_vuelos.drop(['STD', 'STA', 'Aeronave', 'Bookings', 'Destination_Type', 'Origin_Type'], axis=1, inplace=True)

Cantidades de poblaciones de vuelo

In [6]:
# Filtrar los registros con valores nulos en las columnas 'Passengers' y 'Bookings'
df_vuelos_prediccion = df_vuelos[df_vuelos['Passengers'].isnull()]
# Filtrar los registros sin valores nulos en las columnas 'Passengers' y 'Bookings'
df_vuelos = df_vuelos.dropna(subset=['Passengers'])

print("Cantidad de vuelos predectivos: ", len(df_vuelos_prediccion))
print("Cantidad de vuelos de entrenamiento: ", len(df_vuelos))

Cantidad de vuelos predectivos:  43637
Cantidad de vuelos de entrenamiento:  121724


Encodear

In [7]:
# Inicializar el codificador de etiquetas
label_encoder = LabelEncoder()

# Codificar las columnas categóricas
categorical_columns = ['DepartureStation', 'ArrivalStation', 'Departure_WeekDay']
for column in categorical_columns:
    df_vuelos[column] = label_encoder.fit_transform(df_vuelos[column])


Normalización de datos

In [8]:
#Eliminacion de datos atipicos para el entrenamiento del modelo
df_vuelos_prediccion= df_vuelos.copy()
z = np.abs(stats.zscore(df_vuelos['Passengers']))
outliers = df_vuelos[z > 3.49]
df_vuelos = df_vuelos.drop(outliers.index)

Creación de modelo

In [9]:

# Dividir los datos en características (X) y la variable objetivo (y)
X = df_vuelos.drop(columns=['Passengers'])
y = df_vuelos['Passengers']  # Ahora y es un DataFrame con dos columnas

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir el rango de hiperparámetros para la búsqueda de cuadrícula
param_grid = {
    'n_estimators': [197], # Perfecto
    'learning_rate': [0.1622], # Perfecto
    'max_depth': [8], # Perfecto
    'min_samples_split': [2], # Perfecto
    'min_samples_leaf': [4], # Perfecto
} # Para abajo?

# Inicializar el modelo de Gradient Boosting Regressor
gradient_boosting = GradientBoostingRegressor(random_state=42)

# Inicializar GridSearchCV con el modelo y los hiperparámetros
grid_search = GridSearchCV(estimator=gradient_boosting, param_grid=param_grid, cv=3, scoring='neg_mean_absolute_percentage_error', n_jobs=-1, verbose=3, error_score='raise')

# Entrenar GridSearchCV en los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros encontrados
best_params = grid_search.best_params_
print("Mejores hiperparámetros:", best_params)


Fitting 3 folds for each of 1 candidates, totalling 3 fits


Resultados de predicción

In [ ]:

# Obtener el mejor modelo encontrado
best_model = grid_search.best_estimator_

y_true = np.array(y_test)
y_pred = best_model.predict(X_test)
df_predicciones = pd.DataFrame({'Real': y_true, 'Predicción': y_pred}).round(0)
df_predicciones.head(20)

# Calcular la diferencia porcentual de pasajeros entre las predicciones y los valores reales
df_predicciones['Diferencia porcentual de pasajeros'] = abs((df_predicciones['Real'] - df_predicciones['Predicción']) / df_predicciones['Real'])*100

# Mostrar las primeras filas del DataFrame de predicciones
df_predicciones.describe()


,Real,Predicción,Diferencia porcentual de pasajeros
count,24286.000000,24286.000000,24286.000000
mean,184.087499,184.101252,9.644596
std,36.488662,29.862552,12.269364
min,53.000000,55.000000,0.000000
25%,162.000000,165.000000,2.762431
50%,185.000000,182.000000,6.153846
75%,210.000000,207.000000,12.121212
max,262.000000,259.000000,224.137931


Observar características valiosas

In [ ]:

# Obtener el mejor modelo encontrado
best_model = grid_search.best_estimator_

# Obtener la importancia de las características
feature_importance = best_model.feature_importances_

# Crear un DataFrame con la importancia de las características
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importance})

# Ordenar el DataFrame por importancia en orden descendente
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Mostrar el ranking de importancia de características
print(feature_importance_df)


             Feature  Importance
2           Capacity    0.399355
0   DepartureStation    0.160974
1     ArrivalStation    0.106357
7        Flight_Time    0.103391
6    Departure_Month    0.097611
3  Departure_WeekDay    0.057790
4     Departure_Hour    0.038675
5       Arrival_Hour    0.035847


Guardar modelo

In [10]:
# Guardar el modelo en un archivo
joblib.dump('modelo_pasajeros.pkl')

NameError: name 'modelo_pasajeros' is not defined

Predecir en los registros nulos

In [11]:
# Filtrar los registros con valores nulos en la columna 'Passengers'
df_vuelos_prediccion = df_vuelos[df_vuelos['Passengers'].isnull()]
print("Cantidad de vuelos predictivos con valores nulos en 'Passengers': ", len(df_vuelos_prediccion))

# Cargar el modelo desde un archivo
modelo = joblib.load('modelo_pasajeros.pkl')

# Usa el modelo para hacer predicciones
prediccion = modelo.predict(df_vuelos_prediccion)

# Df con las predicciones
df_vuelos_prediccion['Passengers'] = prediccion
df_vuelos_prediccion['Passengers'] = df_vuelos_prediccion['Passengers'].astype(int)

# Head de las predicciones
df_vuelos_prediccion.head()

Cantidad de vuelos predictivos con valores nulos en 'Passengers':  0


c:\Users\RHJ\anaconda3\Lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator DummyRegressor from version 1.4.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


ValueError: node array from the pickle has an incompatible dtype:
- expected: [('left_child', '<i8'), ('right_child', '<i8'), ('feature', '<i8'), ('threshold', '<f8'), ('impurity', '<f8'), ('n_node_samples', '<i8'), ('weighted_n_node_samples', '<f8')]
- got     : {'names': ['left_child', 'right_child', 'feature', 'threshold', 'impurity', 'n_node_samples', 'weighted_n_node_samples', 'missing_go_to_left'], 'formats': ['<i8', '<i8', '<i8', '<f8', '<f8', '<i8', '<f8', 'u1'], 'offsets': [0, 8, 16, 24, 32, 40, 48, 56], 'itemsize': 64}

#Concatenacion con los datos de vuelo

In [ ]:
# Cargar el DataFrame vuelos
df_vuelos = pd.read_csv('Filghts TEC_Valid.csv')

In [ ]:
# Filtrar datos eliminando registros con valores nulos en las columnas especificadas
df_vuelos = df_vuelos.dropna(subset=['Aeronave', 'DepartureStation', 'ArrivalStation', 'Destination_Type', 'Origin_Type'])

# Convertir la columna 'STD' a formato datetime
df_vuelos['STD'] = pd.to_datetime(df_vuelos['STD'])
df_vuelos['STA'] = pd.to_datetime(df_vuelos['STA'])
df_vuelos['Departure_WeekDay'] = df_vuelos['STD'].dt.day_name()

# Extraer la hora y el mes de la columna 'STD'
df_vuelos['Departure_Hour'] = df_vuelos['STD'].dt.hour
df_vuelos['Arrival_Hour'] = df_vuelos['STA'].dt.hour
df_vuelos['Departure_Month'] = df_vuelos['STD'].dt.month
# Calcular la duración del vuelo restando STA de STD
df_vuelos['Flight_Time'] = (pd.to_datetime(df_vuelos['STA']) - pd.to_datetime(df_vuelos['STD'])).dt.total_seconds() / 60

# Convertir el tiempo de vuelo a entero
df_vuelos['Flight_Time'] = df_vuelos['Flight_Time'].astype(int)
# Convertir la hora de salida y el mes a entero
df_vuelos['Departure_Hour'] = df_vuelos['Departure_Hour'].astype(int)
df_vuelos['Departure_Month'] = df_vuelos['Departure_Month'].astype(int)

# Eliminar columnas 'STD', 'STA' y 'Bookings'
df_vuelos.drop(['STD', 'STA', 'Aeronave', 'Bookings', 'Destination_Type', 'Origin_Type'], axis=1, inplace=True)

# Inicializar el codificador de etiquetas
label_encoder = LabelEncoder()

# Codificar las columnas categóricas
categorical_columns = ['DepartureStation', 'ArrivalStation', 'Departure_WeekDay']
for column in categorical_columns:
    df_vuelos[column] = label_encoder.fit_transform(df_vuelos[column])

# Head del DataFrame
df_vuelos.head()

<ipython-input-68-fc09fa51cf0c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vuelos['STD'] = pd.to_datetime(df_vuelos['STD'])
<ipython-input-68-fc09fa51cf0c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vuelos['STA'] = pd.to_datetime(df_vuelos['STA'])
<ipython-input-68-fc09fa51cf0c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

,Flight_ID,DepartureStation,ArrivalStation,Capacity,Passengers,Departure_WeekDay,Departure_Hour,Arrival_Hour,Departure_Month,Flight_Time
0,ab954014077430bd842cfa305a55c0f8,11,18,240.0,229.0,4,11,14,10,165
1,efd86c996035dacdca7a0ccb2560dda1,26,16,186.0,197.0,1,0,4,7,240
2,6cfa1bbaa44f08fc7d3061f034a6a5ce,13,14,220.0,NaN,0,17,17,2,45
3,dd0fad3248951d2f71d63e6279aeaa4b,13,14,220.0,200.0,1,15,15,6,40
4,d0987ee648eea254063bfe2b39571b67,16,0,186.0,162.0,0,8,9,2,70


In [ ]:
# Filtrar los registros con valores nulos en las columnas 'Passengers' y 'Bookings'
df_vuelos_prediccion = df_vuelos[df_vuelos['Passengers'].isnull()]
# Filtrar los registros sin valores nulos en las columnas 'Passengers' y 'Bookings'
df_vuelos = df_vuelos.dropna(subset=['Passengers'])

print("Cantidad de vuelos predectivos: ", len(df_vuelos_prediccion))
print("Cantidad de vuelos de entrenamiento: ", len(df_vuelos))

# Eliminar registros con valores nulos en la columna 'Passengers' del DataFrame original
df_vuelos_prediccion = df_vuelos_prediccion.drop(columns=['Passengers'])

# Flight ID's vuelos predictivos
flight_ids = df_vuelos_prediccion['Flight_ID']
# Eliminar columna 'Flight_ID'
df_vuelos_prediccion.drop(columns=['Flight_ID'], inplace=True)

# Cargar el modelo desde un archivo
modelo = joblib.load('/content/modelo_pasajeros.pkl')

# Predecir la cantidad de pasajeros
predicciones = modelo.predict(df_vuelos_prediccion)

# Asignar las predicciones al DataFrame de vuelos predictivos
df_vuelos_prediccion['Passengers'] = predicciones

# Asignar los Flight ID's a las predicciones
df_vuelos_prediccion['Flight_ID'] = flight_ids

# Unir los DataFrames
df_vuelos = pd.concat([df_vuelos, df_vuelos_prediccion])

# Remover dataframes temporales
del df_vuelos_prediccion

# Cantidad de vuelos en el DataFrame
print("Cantidad de vuelos en el DataFrame: ", len(df_vuelos))
# head del DataFrame
df_vuelos.head()


Cantidad de vuelos predectivos:  17516
Cantidad de vuelos de entrenamiento:  53647
Cantidad de vuelos en el DataFrame:  71163


,Flight_ID,DepartureStation,ArrivalStation,Capacity,Passengers,Departure_WeekDay,Departure_Hour,Arrival_Hour,Departure_Month,Flight_Time
0,ab954014077430bd842cfa305a55c0f8,11,18,240.0,229.0,4,11,14,10,165
1,efd86c996035dacdca7a0ccb2560dda1,26,16,186.0,197.0,1,0,4,7,240
3,dd0fad3248951d2f71d63e6279aeaa4b,13,14,220.0,200.0,1,15,15,6,40
4,d0987ee648eea254063bfe2b39571b67,16,0,186.0,162.0,0,8,9,2,70
5,3b5df8805161ea827d2f2e4298c38e06,4,13,240.0,183.0,4,17,18,9,55


In [ ]:
# Cargar el DataFrame especificando los tipos de datos
df_ventas = pd.read_csv('Sales TEC_Valid.csv', dtype={'Quantity': 'Int64'})

# Lista de cadenas para filtrar
categorias_mantener = ['Botanas', 'Galletas', 'Bebidas Calientes', 'Refrescos', 'Sopas', 'Perecederos']

# Filtrar el DataFrame y eliminar la columna 'TotalSales'
df_ventas = df_ventas[df_ventas['ProductType'].isin(categorias_mantener)].drop(columns=['TotalSales'])

# Eliminar la columna 'ProductType'
df_ventas.drop(columns=['ProductType'], inplace=True)

# Verificar y eliminar duplicados en el DataFrame original
df_ventas = df_ventas.drop_duplicates(subset=['Flight_ID', 'ProductName'])

# Obtener todos los nombres de productos únicos
productos_unicos = df_ventas['ProductName'].unique()

# Crear DataFrame con todas las combinaciones posibles de Flight_ID y ProductName
flight_ids = df_ventas['Flight_ID'].unique()
combinaciones = []
for flight_id in flight_ids:
    for producto in productos_unicos:
        combinaciones.append({'Flight_ID': flight_id, 'ProductName': producto})
df_combinaciones = pd.DataFrame(combinaciones)

# Fusionar con el DataFrame original para obtener las combinaciones faltantes
df_faltantes = pd.merge(df_combinaciones, df_ventas, on=['Flight_ID', 'ProductName'], how='left')

# Filtrar solo las combinaciones faltantes
df_faltantes = df_faltantes[df_faltantes['Quantity'].isnull()]
df_faltantes['Quantity'] = 0

# Concatenar las combinaciones faltantes al DataFrame principal
df_ventas = pd.concat([df_ventas, df_faltantes], ignore_index=True)

# Verificar y eliminar duplicados en el DataFrame combinado
df_ventas = df_ventas.drop_duplicates(subset=['Flight_ID', 'ProductName'])

# Ordenar el DataFrame por Flight_ID y ProductName
df_ventas = df_ventas.sort_values(by=['Flight_ID', 'ProductName']).reset_index(drop=True)

# Resetear los índices del DataFrame
df_ventas.reset_index(drop=True, inplace=True)

# Cantidad de registros en el DataFrame
print("Cantidad de registros en el DataFrame: ", len(df_ventas))

<ipython-input-70-3eaee949c302>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_faltantes['Quantity'] = 0


Cantidad de registros en el DataFrame:  7481601


In [ ]:

df_vuelos = df_vuelos.drop_duplicates(subset=['Flight_ID'])
df_vuelos

,Flight_ID,DepartureStation,ArrivalStation,Capacity,Passengers,Departure_WeekDay,Departure_Hour,Arrival_Hour,Departure_Month,Flight_Time
0,ab954014077430bd842cfa305a55c0f8,11,18,240.0,229.000000,4,11,14,10,165
1,efd86c996035dacdca7a0ccb2560dda1,26,16,186.0,197.000000,1,0,4,7,240
3,dd0fad3248951d2f71d63e6279aeaa4b,13,14,220.0,200.000000,1,15,15,6,40
4,d0987ee648eea254063bfe2b39571b67,16,0,186.0,162.000000,0,8,9,2,70
5,3b5df8805161ea827d2f2e4298c38e06,4,13,240.0,183.000000,4,17,18,9,55
...,...,...,...,...,...,...,...,...,...,...
102434,12efd56dee9100b9f9ae879cc3b952e6,4,15,240.0,203.553629,6,7,10,1,170
102439,301a862b6fd449ad0dbb139f438d0ba9,12,15,240.0,237.315315,0,9,11,4,125
102440,f5f0c5190dd56f44d7bf739d816b8f9a,13,15,220.0,211.885736,4,18,20,2,100
102441,652580e513cceefd667cb92702325939,28,15,220.0,214.807655,4,8,9,3,100


In [ ]:
# Obtener todos los productos únicos de df_ventas
productos_unicos = df_ventas['ProductName'].unique()

# Crear un DataFrame con todos los vuelos
df_vuelos_completo = pd.DataFrame({'Flight_ID': df_vuelos['Flight_ID'].unique()})

# Crear un DataFrame con todos los productos únicos
df_productos = pd.DataFrame({'ProductName': productos_unicos})

# Hacer una combinación de cada Flight_ID con cada producto único
df_vuelos_completo = pd.merge(df_vuelos_completo, df_productos, how='cross')

# Agregar la columna Quantity con valor 0
df_vuelos_completo['Quantity'] = 0

# Resetear los índices del DataFrame
df_vuelos_completo.reset_index(drop=True, inplace=True)

# Mostrar las primeras filas del DataFrame resultante
df_vuelos_completo['Quantity'].value_counts()

Quantity
0    4899690
Name: count, dtype: int64

In [ ]:
# Concatenar df_ventas y df_vuelos_completo
df_vuelos_ventas = pd.concat([df_ventas, df_vuelos_completo], ignore_index=True)

# Cantidad de registros en el DataFrame
print("Cantidad de registros en el DataFrame: ", len(df_vuelos_ventas))

# Mostrar el resultado
df_vuelos_ventas["Flight_ID"].value_counts()

Cantidad de registros en el DataFrame:  12381291


Flight_ID
00004a718edba9d9ef878d08f02ae057    138
7ac84ff2396f5527364887b46dc4cfc1    138
f29d01dffcbefd9007cc8d8ba03a16aa    138
f29a222c42820aa80f1fbcd05aa094da    138
f299c2a26bea8847e7a34725f7a665e4    138
                                   ... 
854032a44310795d38ffeb9f6c9c9d67     69
854003fa25c7b9cf8edaee7b9014edd6     69
853ededf991996d025eb8e6215649aa8     69
853d1eb8483aee2982516e554dfcb898     69
e76b6a4666a0b75408b37d209d5125eb     69
Name: count, Length: 131942, dtype: int64

In [ ]:
aztest_complete_df = pd.merge(df_vuelos, df_vuelos_completo, on='Flight_ID', how='inner')

test_complete_df

,Flight_ID,DepartureStation,ArrivalStation,Capacity,Passengers,Departure_WeekDay,Departure_Hour,Arrival_Hour,Departure_Month,Flight_Time,ProductName,Quantity
0,ab954014077430bd842cfa305a55c0f8,11,18,240.0,229.000000,4,11,14,10,165,Agua Natural 600 Ml,0
1,ab954014077430bd842cfa305a55c0f8,11,18,240.0,229.000000,4,11,14,10,165,Arandano,0
2,ab954014077430bd842cfa305a55c0f8,11,18,240.0,229.000000,4,11,14,10,165,Arandano Mango Mix,0
3,ab954014077430bd842cfa305a55c0f8,11,18,240.0,229.000000,4,11,14,10,165,Arcoiris,0
4,ab954014077430bd842cfa305a55c0f8,11,18,240.0,229.000000,4,11,14,10,165,Cafe 19 Cafe Clasico,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4899685,e76b6a4666a0b75408b37d209d5125eb,9,15,230.0,214.409337,0,16,17,4,95,Te Manzanilla Jengibre,0
4899686,e76b6a4666a0b75408b37d209d5125eb,9,15,230.0,214.409337,0,16,17,4,95,Te Relax,0
4899687,e76b6a4666a0b75408b37d209d5125eb,9,15,230.0,214.409337,0,16,17,4,95,Te Vainilla,0
4899688,e76b6a4666a0b75408b37d209d5125eb,9,15,230.0,214.409337,0,16,17,4,95,Tostitos,0


In [ ]:
test_complete_df.to_csv("Complete_VIVA_DF_2023.csv")